In [1]:
# This is the part 2 that will go from steps 10 until 13.
""" 
1- Read data into a dataframe (mma)
2- Separate data into train/test (80/20) (mma_train , mma_test)
3- Calculate frequency in train data | test data of each product over all orders (mma_train_freq | mma_test_freq)
4- Create a solution table with columns (ProductID|SubstituteID|PFrequency|DepartmentID|AisleID|Selected?) (solution_df)
5- Populate the solution table ProductID column with a {SELECT DISTINCT ProductID} over original data
6- Add information of product frequency on orders per ProductID to solution table (PFrequency), by joining with mma_train_freq
7- Add DepartmentID|AisleID to solution table by joining with main table
8- Create SubstituteIDs for each ProductID
    >> For each AisleID|DepartmentID group:
        1)Select all ProductIDs | ProductNames under Aisle|Department
        2)Do a similarity analysis, and try to cluster similar product names
        3)On each cluster, compare frequency of product in orders, and select the most frequent product as SubstituteID
            Left Join with frequency table, and compare
        4)For outliers (products that are not similar to any other product), copy ProductID as SubstituteID
        5)Append ProductIDs|SubstituteIDs to Substitute table
9- Add SubstituteIDs to solution table by joining with Substitute table
10- Analyze products per frequency in orders to select those with highest probability to select, considering constraints
11- Repeat the analysis but accounting for substitute products, selecting products based on potential substitute presence
12- Test solution table with test data
13- Calculate score against metrics
"""

' \n1- Read data into a dataframe (mma)\n2- Separate data into train/test (80/20) (mma_train , mma_test)\n3- Calculate frequency in train data | test data of each product over all orders (mma_train_freq | mma_test_freq)\n4- Create a solution table with columns (ProductID|SubstituteID|PFrequency|DepartmentID|AisleID|Selected?) (solution_df)\n5- Populate the solution table ProductID column with a {SELECT DISTINCT ProductID} over original data\n6- Add information of product frequency on orders per ProductID to solution table (PFrequency), by joining with mma_train_freq\n7- Add DepartmentID|AisleID to solution table by joining with main table\n8- Create SubstituteIDs for each ProductID\n    >> For each AisleID|DepartmentID group:\n        1)Select all ProductIDs | ProductNames under Aisle|Department\n        2)Do a similarity analysis, and try to cluster similar product names\n        3)On each cluster, compare frequency of product in orders, and select the most frequent product as Substit

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import plotly.express as px
import scipy as sp
import sklearn as sk
import seaborn as sns
import re
import string
import math
import random
import itertools
import time
import pulp
import os
import warnings
import string
warnings.filterwarnings('ignore')


In [3]:
### Read data from Part 1
mma_test = pd.read_csv('data/processed/to_optimizer/mma_test.csv')
mma_train = pd.read_csv('data/processed/to_optimizer/mma_train.csv')
solution_df = pd.read_csv('data/processed/to_optimizer/solution_df.csv')

In [4]:
selected = pd.read_csv('data/results/selections.csv')
substitutes = pd.read_csv('data/results/substitutes.csv')
selected_nosubstitute = pd.read_csv('data/results/selections_nosubstitute.csv')

### 10- Prepare objective function as a method

In [5]:
def metric1(mma_train, solution_df):
    """ 
    Metric 1 : Number of orders that use in instabasket aisle items
    """
    # Select all orders that use in-aisle items
    
    
    return m1

def metric2(mma_train, solution_df):
    """ 
    Metric 1 : Number of orders that use in instabasket aisle items
    """
    # Select all orders that use in-aisle items
    
    
    return m2

### 11- Prepare constraints as methods

In [6]:
def freezer_constraint(df):
    """
    The freezer constraint limits the amount of freezer products on the resulting list to a maximum of 100
    """

    if df.loc[df['department'] == 'frozen']['Selected?'].sum() <= 100:
        return True
    else:
        return False
    
def refrigerator_constraint(df):
    """
    The refrigerator constraint limits the amount of refrigerated products on the resulting list to a maximum of 100
    """
    if df.loc[(df['department'] == 'meat seafood' ) | (df['department'] == 'dairy eggs')]['Selected?'].sum() <= 100:
        return True
    else:
        return False

def selections_constraint(df):
    """
    The selections constraint limits the amount of selections resulting list to a maximum of 1000
    """
    if df['Selected?'].sum() <= 1000:
        return True
    else:
        return False

In [7]:
selected

,product_id,department_name,product_name
0,27845,dairy eggs,Organic Whole Milk
1,33754,dairy eggs,Total 2% with Strawberry Lowfat Greek Strained...
2,49235,dairy eggs,Organic Half & Half
3,5785,dairy eggs,Organic Reduced Fat 2% Milk
4,35951,dairy eggs,Organic Unsweetened Almond Milk
...,...,...,...
995,23094,canned goods,Organic Lentil Soup
996,12107,deli,Peppered Turkey Breast
997,39468,babies,"Apples, Kale & Avocados Organic Baby Food"
998,38662,other,Roasted Almond Butter


### 15- Test solution table with test data

In [8]:
### METHODS FROM MMA_MART_TEMPLATE for testing ###

# Methods for testing the solution
def tot_order(mma_test):
    # calculate total unique orders in the test data
    tot_order = mma_test['order_id'].nunique()
    print(f"Total number of orders: {tot_order}")
    return tot_order

def avg_item(mma_test, tot_order):
    # calculate the average number of items in each order
    avg_item = len(mma_test) / tot_order
    print(f"Average number of items in each order: {avg_item}")
    return avg_item

def test_my_pick(mma_test, df):
    # join the test data with my pick on 'product_id'
    test_my_pick = mma_test.merge(df, left_on='product_id', right_on='product_id', how='left')

    return test_my_pick

def metrics_tmp(test_my_pick):
    # group by 'order_id' and count the non-null values of 'product_name_y' column
    metrics_tmp = test_my_pick[['order_id', 'product_name_y']].groupby('order_id').count().reset_index().rename(columns={'product_name_y': "num_prod_matching"})
    
    return metrics_tmp

def metric1(metrics_tmp, tot_order):
    # count orders with at least a match
    metric_1 = len(metrics_tmp[metrics_tmp['num_prod_matching']>0])
    print(f"Out of {tot_order} orders, {metric_1} of them utilize the in-aisle items. That's about {metric_1/tot_order*100:.2f}%.")
    return metric1

def metric2(metrics_tmp, avg_item):
    # calculate metric 2
    metric_2 = metrics_tmp['num_prod_matching'].sum() / len(metrics_tmp)
    print(f"On average there are {avg_item:.2f} items in each order, and {metric_2:.2f} of them utilize in-aisle items. That's about {metric_2/avg_item*100:.2f}%")
    return metric_2

def test(mma_test, df):
    tot_ord = tot_order(mma_test)
    avg_it = avg_item(mma_test, tot_ord)
    test_my_pic = test_my_pick(mma_test, df)
    metrics_tp = metrics_tmp(test_my_pic)

    metric_1 = metric1(metrics_tp, tot_ord)
    metric_2 = metric2(metrics_tp, avg_it)

    return metric_1, metric_2

In [9]:
metric_1, metric_2 = test(mma_test, df = selected)

Total number of orders: 19584
Average number of items in each order: 10.136540032679738
Out of 19584 orders, 17425 of them utilize the in-aisle items. That's about 88.98%.
On average there are 10.14 items in each order, and 4.13 of them utilize in-aisle items. That's about 40.77%


In [10]:
metric_3, metric_4 = test(mma_test, df = selected_nosubstitute)

Total number of orders: 19584
Average number of items in each order: 10.136540032679738
Out of 19584 orders, 18058 of them utilize the in-aisle items. That's about 92.21%.
On average there are 10.14 items in each order, and 5.31 of them utilize in-aisle items. That's about 52.43%


### Accounting for Substitutes

In [11]:
### METHODS FROM MMA_MART_TEMPLATE for testing accounting for Substitutes ###

# Method to account for substitutes, we will consider the product_id in mma_test to be the substituted now.
def substitute_ids(mma_test, substitutes):
    mma_test_sub = mma_test.merge(substitutes, left_on='product_id', right_on='product_id', how='left')
    mma_test_sub.rename(columns={'product_id':'product_id_legacy'}, inplace=True)
    mma_test_sub.rename(columns={'product_id2':'product_id'}, inplace=True)
    mma_test_sub.head()
    
    return mma_test_sub

In [12]:
mma_test_sub = substitute_ids(mma_test, substitutes)
metric_5, metric_6 = test(mma_test_sub, df = selected)

Total number of orders: 19584
Average number of items in each order: 10.136540032679738
Out of 19584 orders, 19136 of them utilize the in-aisle items. That's about 97.71%.
On average there are 10.14 items in each order, and 7.63 of them utilize in-aisle items. That's about 75.23%
